# 第二章第六节：Testers2

## 本节的目的
Chisel团队一直在改进测试框架，新的测试框架我们称为“Testers2”，它提供了以下改进：

- 适用于单元测试和系统集成测试
- 设计考虑了可组合的抽象和分层
- 高度可用，尽可能使得单元测试简单，方便（避免不重要，但是又没它不行的代码），并有用
 
### 计划中的特性
- 适用于多个后端和仿真器（如果测试向量不是静态的可能需要链接到Scala，或者综合到FPGA时使用一定的构建测试的API）
- 将包含在基本的chisel3中，以避免额外的包和其他的依赖关系


## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.experimental._
import chisel3.experimental.BundleLiterals._
import chisel3.tester._
import chisel3.tester.RawTester.test

>这一节您可能注意到上面的导入和其他节不太一样。
`import chisel3.tester.RawTester.test`引入
下面版本的`test(...)`，这是专门为本教程设计的。

---
# 基本测试的实现

Testers2与iotester的基本操作相同。下面是基本功能的简要总结，对比旧的iotesters和新的testers2：


|  操作    | iotesters             | testers2            |
| :----  | :---                  | :---                |
| poke   | poke(c.io.in1, 6)     | c.io.in1.poke(6.U)    |
| peek   | peek(c.io.out1)       | c.io.out1.peek(6.U)   |
| expect | expect(c.io.out1, 6)  | c.io.out1.expect(6.U) |
| step   | step(1)               | c.io.clock.step(1)  |
| initiate | Driver.execute(...) { c => | test(...) { c => |



让我们从2.1节中简单的旁路电路模块开始看起：

In [3]:
// Chisel 代码，不同的是传入一个参数来设置端口的宽度
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

defined class PassthroughGenerator

使用旧的测试方法，看起来像这样：

```scala
val testResult = Driver(() => new Passthrough()) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)     // 设置输入为 0
    expect(c.io.out, 0)  // 检查正确的输出为 0
    poke(c.io.in, 1)     // 设置输入为 1
    expect(c.io.out, 1)  // 检查正确的输出为 1
    poke(c.io.in, 2)     // 设置输入为 2
    expect(c.io.out, 2)  // 检查正确的输出为 2
  }
}
assert(testResult)   // Scala 代码: 如果 testResult == false, 抛出错误异常
println("SUCCESS!!") // Scala 代码: 如果到这里，测试通过！
```



In [4]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // 设置输入为 0
    c.io.out.expect(0.U)  // 检查正确的输出为 0
    c.io.in.poke(1.U)     // 设置输入为 1
    c.io.out.expect(1.U)  // 检查正确的输出为 1
    c.io.in.poke(2.U)     // 设置输入为 2
    c.io.out.expect(2.U)  // 检查正确的输出为 2
}

[info] [0.001] Elaborating design...
[info] [0.624] Done elaborating.
Total FIRRTL Compile Time: 321.7 ms
Total FIRRTL Compile Time: 86.7 ms
file loaded in 0.244742 seconds, 5 symbols, 2 statements
test cmd2HelperPassthroughGenerator Success: 0 tests passed in 2 cycles in 0.061215 seconds 32.67 Hz


>下面只是为了说明testers2怎样往前推进时钟
下面的例子中添加了一些`step`操作。

In [5]:
test(new PassthroughGenerator(16)) { c =>
    c.io.in.poke(0.U)     // 设置输入为 0
    c.clock.step(1)    // 前进一个时钟周期
    c.io.out.expect(0.U)  // 检查正确的输出为 0
    c.io.in.poke(1.U)     // 设置输入为 1
    c.clock.step(1)    // 前进一个时钟周期
    c.io.out.expect(1.U)  // 检查正确的输出为 1
    c.io.in.poke(2.U)     // 设置输入为 2
    c.clock.step(1)    // 前进一个时钟周期
    c.io.out.expect(2.U)  // 检查正确的输出为 2
}

[info] [0.000] Elaborating design...
[info] [0.088] Done elaborating.
Total FIRRTL Compile Time: 12.8 ms
Total FIRRTL Compile Time: 33.6 ms
file loaded in 0.038107 seconds, 5 symbols, 2 statements
test cmd2HelperPassthroughGenerator Success: 0 tests passed in 5 cycles in 0.010393 seconds 481.09 Hz


---
## 上面的例子中需要注意什么

Testers2测试方法去掉了一些不重要，但是又没它不行的代码。这在旧的`PeekPokeTester`中是做不到的。

`poke`和`expect`方法现在是每个`io`元素的一部分。
这可以让测试更好地检查类型。
`peek`和`step`方法现在也是`io`元素的方法了。

另一个区别是值`poked`和`expected`现在是Chisel的字面值（literal）了。
虽然这里的例子非常简单，但它为更高级和有趣的例子提供了更强大的检查。
接下来的改进将支持指定`Bundle`字面值（literal），使得这个能力将进一步增强。

# 带有`Decoupled`接口的模块
在本节中，我们将介绍一些用于处理`Decoupled`接口的tester2工具。
`Decoupled`可以将基本的chisel数据类型包装起来，并为其提供`ready`和`valid`信号。
Testers2提供了一些很好的工具，可以自动并可靠地测试这些接口。

## 队列示例
下面的`QueueModule`传递类型为`ioType`的数据。 `QueueModule`的大小为`entries`个元素，这是它在报告backpressure之前可以保存的元素的数目。

In [6]:
case class QueueModule[T <: Data](ioType: T, entries: Int) extends MultiIOModule {
  val in = IO(Flipped(Decoupled(ioType)))
  val out = IO(Decoupled(ioType))
  out <> Queue(in, entries)
}


defined class QueueModule

> 注意上面`case`的修饰符通常不是必需的，在这里存在是为了在Jupyter的多个单元格中重复使用这个例子

## EnqueueNow 和 expectDequeueNow
*testers2*有一些内置的方法来处理IO中具有`Decoupled`接口的电路。在这个例子中，我们将看到如何从`queue`中插入和提取值。

| 方法 | 描述 |
| :---   | :---        |
| enqueueNow | 往`Decoupled`输入接口中添加一个元素 (入队) |
| expectDequeueNow | 从`Decoupled`输出接口中移除一个元素（出列） |
---


>注意：有一些必要的但不重要的代码，如`initSource`，`setSourceClock`等，这是为了确保`ready`和`valid`在测试开始时都被正确初始化。

In [7]:
test(QueueModule(UInt(9.W), entries = 200)) { c =>
    //示例测试了队列 Queue 的用法和行为
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(200){ i => i.U }

    testVector.zip(testVector).foreach { case (in, out) =>
      c.in.enqueueNow(in)
      c.out.expectDequeueNow(out)
    }
}

[info] [0.000] Elaborating design...
[info] [0.158] Done elaborating.
Total FIRRTL Compile Time: 148.3 ms
Total FIRRTL Compile Time: 109.2 ms
file loaded in 0.200071 seconds, 63 symbols, 54 statements
test cmd5HelperQueueModule Success: 0 tests passed in 402 cycles in 0.340215 seconds 1181.61 Hz


## EnqueueSeq 和 DequeueSeq 
Now we are going to introduce two new methods that deal with enqueuing and dequeuing operations in single operations.

现在我们将介绍两种新方法，用于处理单个操作中的入队和出列操作。

| 方法 | 描述 |
| :---   | :---        |
| enqueueSeq | 陆续将`Seq`中的元素依次添加（入队）到`Decoupled`输入接口中，一次一个，直到序列耗尽 |
| expectDequeueSeq | 依次从一个`Decoupled`输出接口中移除（出列）元素，一次一个，并将每个元素与`Seq`中元素依次进行比较 |

---

>注意：下面的例子工作正常但是，如上所述，`enqueueSeq`必须在`expectDequeueSeq`开始之前完成。如果`testVector`的大小大于队列的深度，那么测试会失败，因为队列将被填满并且无法完成入队操作`enqueueSeq`。建议您改一下代码来看看测试如何会失败。在下一节中，我们将展示如何正确地构建此类测试。


In [8]:
test(QueueModule(UInt(9.W), entries = 200)) { c =>
    // 示例测试了队列 Queue 的用法和行为
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(100){ i => i.U }

    c.in.enqueueSeq(testVector)
    c.out.expectDequeueSeq(testVector)
}

[info] [0.000] Elaborating design...
[info] [0.078] Done elaborating.
Total FIRRTL Compile Time: 43.4 ms
Total FIRRTL Compile Time: 50.7 ms
file loaded in 0.069079 seconds, 63 symbols, 54 statements
test cmd5HelperQueueModule Success: 0 tests passed in 202 cycles in 0.132720 seconds 1522.00 Hz


> 刚才这一小节中另外一个重点是我们刚刚看到的函数`enqueueNow`，
`enqueueSeq`，`expectDequeueNow`和`expectDequeueSeq`在testers2中并不是复杂特殊的逻辑。
相反，它们是鼓励您使用的testers2的基本元素。要了解这些方法是如何实现的，请查看[TestAdapters.scala](https://github.com/ucb-bar/chisel-testers2/blob/d199c5908828d0be5245f55fce8a872b2afb314e/src/main/scala/chisel3/tester/TestAdapters.scala)

# testers2中的Fork和Join

在这一小节中，我们将看到如何在单元测试中并发地执行代码。为此，我们将介绍testers22的两个新功能：

| 方法 | 描述 |
| :---   | :---        |
| fork   | 启动一个并发代码块，可以通过在前一个fork的代码块末尾附加.fork来并发执行另外的fork代码块 |
| join | 将多个fork的分叉重新整合并返回调用线程 |

---

在下面的例子中，有两个连在一起的`fork`，在最后被`join`起来。在第一个`fork`块中，`enqueueSeq`将依次添加元素直到耗尽。每当数据可用时，第二个`fork`块将调用`expectDequeueSeq`在每个时钟周期检查输出。

> fork创建的线程以确定的顺序运行，主要根据代码中指定的顺序运行，并且运行时检查禁止某些依赖于其他线程的易受错误的操作。

In [9]:

test(QueueModule(UInt(9.W), entries = 200)) { c =>
    // 示例测试了队列 Queue 的用法和行为
    c.in.initSource()
    c.in.setSourceClock(c.clock)
    c.out.initSink()
    c.out.setSinkClock(c.clock)
    
    val testVector = Seq.tabulate(300){ i => i.U }

    fork {
        c.in.enqueueSeq(testVector)
    }.fork {
        c.out.expectDequeueSeq(testVector)
    }.join()
}

[info] [0.000] Elaborating design...
[info] [0.085] Done elaborating.
Total FIRRTL Compile Time: 49.3 ms
Total FIRRTL Compile Time: 45.2 ms
file loaded in 0.059423 seconds, 63 symbols, 54 statements
test cmd5HelperQueueModule Success: 0 tests passed in 303 cycles in 0.274249 seconds 1104.84 Hz


## 在GCD模块中使用Fork和Join
在本节中，我们将使用fork join方法来实现*最大公约数* **GCD**的测试。
让我们从定义IO的bundle开始。我们将在这里添加一些必要但不重要的代码，以允许我们使用`Bundle` *字面值（literal）*。正如注释所说，希望我们很快能支持自动生成这些代码。

In [10]:
class GcdInputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
}

defined class GcdInputBundle

In [11]:
class GcdOutputBundle(val w: Int) extends Bundle {
  val value1 = UInt(w.W)
  val value2 = UInt(w.W)
  val gcd    = UInt(w.W)
}

defined class GcdOutputBundle

现在让我们来看看*Decoupled*版本的**GCD** 。我们在这里修改了一些地方，使用`Decoupled`来包装输入输出，在输入和输出`Bundle`中添加`ready`和`valid`信号。 `Flipped`用来包装`Decoupled`接口`GcdInputBundle`，`GcdInputBundle`在默认情况下被创建为输出，这里将每个字段的方向（递归地）转换为相反的方向。 `Decoupled`接口中的数据元素可以通过顶层的字段`bits`来访问。

In [12]:
/**
  * 使用减法来计算GCD。
  * 从寄存器 x 和 y 中较大一个中减去较小的一个，直到寄存器 y 为零。
  * 输入寄存器 x 的值就是最大公约数
  * 返回包含两个输入值及其GCD的包
  */

class DecoupledGcd(width: Int) extends MultiIOModule {

  val input = IO(Flipped(Decoupled(new GcdInputBundle(width))))
  val output = IO(Decoupled(new GcdOutputBundle(width)))

  val xInitial    = Reg(UInt())
  val yInitial    = Reg(UInt())
  val x           = Reg(UInt())
  val y           = Reg(UInt())
  val busy        = RegInit(false.B)
  val resultValid = RegInit(false.B)

  input.ready := ! busy
  output.valid := resultValid
  output.bits := DontCare

  when(busy)  {
    // 在计算期间，从每次较大的值中减去较小的值
    when(x > y) {
      x := x - y
    }.otherwise {
      y := y - x
    }
    when(y === 0.U) {
      // 当 y 变为零时，计算结束，将输出置为 valid
      output.bits.gcd := x
      output.bits.value1 := xInitial
      output.bits.value2 := yInitial
      output.bits.gcd := x
      output.valid := true.B
      busy := false.B
    }
  }.otherwise {
    when(input.valid) {
      // 当有可用的有效数据且没有进行计算时，获取新值并开始计算
      val bundle = input.deq()
      x := bundle.value1
      y := bundle.value2
      xInitial := bundle.value1
      yInitial := bundle.value2
      busy := true.B
      resultValid := false.B
    }
  }
}


defined class DecoupledGcd

我们的测试与之前的Queue测试看起来非常相似。
但是还包含更多事情，因为GCD计算需要多个时钟周期，因此在计算每个GCD时会阻塞住输入的入队。
好消息是，在不同的Decoupled电路中，测试简单且一致。

这里还介绍了新的Chisel3 `Bundle`字面值（literal）。考虑一下这行代码：
```scala
new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
```
上面定义的`GcdInputBundle`有两个字段`value1`和`value2`。
我们首先创建一个bundle然后调用它的`.Lit`方法来创建一个bundle的字面值（literal）。
该方法的输入是一对对的键/值的可变参数列表，其中键（例如 `_.value`）是字段名称，值（例如 x.U）是chisel的硬件字面值（literal），Scala的`Int` x被转换为Chisel的`UInt`字面值（literal）。
字段名称前面的 `_.` 是必须的。

>这可能不是最完美方式，但在广泛的讨论中，它被视为在
最小化必要但不重要的代码与Scala中可用的符号限制之间取得的最佳平衡。


In [13]:
test(new DecoupledGcd(16)) { dut =>
  dut.input.initSource().setSourceClock(dut.clock)
  dut.output.initSink().setSinkClock(dut.clock)

  val testValues = for { x <- 1 to 10; y <- 1 to 10} yield (x, y)
  val inputSeq = testValues.map { case (x, y) =>
    (new GcdInputBundle(16)).Lit(_.value1 -> x.U, _.value2 -> y.U)
  }
  val resultSeq = testValues.map { case (x, y) =>
    new GcdOutputBundle(16).Lit(_.value1 -> x.U, _.value2 -> y.U, _.gcd -> BigInt(x).gcd(BigInt(y)).U)
  }

  fork {
    dut.input.enqueueSeq(inputSeq)
  }.fork {
    dut.output.expectDequeueSeq(resultSeq)
  }.join()
}


[info] [0.000] Elaborating design...
[info] [0.085] Done elaborating.
Total FIRRTL Compile Time: 44.2 ms
Total FIRRTL Compile Time: 34.0 ms
file loaded in 0.045373 seconds, 39 symbols, 33 statements
test cmd11HelperDecoupledGcd Success: 0 tests passed in 664 cycles in 0.374501 seconds 1773.03 Hz


---
# 本节结束!

[返回顶部](#top)